In [2]:
# Core Python Libraries
import os
import copy
import random
from collections import OrderedDict
from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import datetime

# Data Processing Libraries
import numpy as np
import pandas as pd

# PyTorch Libraries
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler, Sampler

# Progress Tracker
from tqdm import tqdm

In [3]:
import datetime
# Define file_path
input_file_path = "/N/lustre/project/proj-212/praval/Global_sediment_flux/lstm_model/codes/cluster/Hydrogeosciences_codes/input_data_sample_COMID_74068275.0.csv"
data_dir = os.path.dirname(input_file_path)

# Function to read and process a single file
def read_file(file_name, data_dir):
    new_data_file = os.path.join(data_dir, file_name)
    df = pd.read_csv(new_data_file)

    COMID = int(df.iloc[0, 0])  #COMID is the first column

    # Process dates
    dates = [datetime.date(*map(int, date_str.split('-'))).toordinal() for date_str in df.iloc[:, 1]]

    # Extract data (All columns after the second are numerical data)
    data = df.iloc[:, 2:].values

    # Select COMID with at least a certain number of observations
    valid_observations = np.count_nonzero(
        [str(val).strip().lower() != 'nan' for val in data[:, 0]]
    )
    
    if valid_observations < 50:
        return None  # Skip files with insufficient valid observations

    return COMID, np.array(dates), torch.tensor(data, dtype=torch.float32)

file_name = os.path.basename(input_file_path)

# Dictionary to store data
data_dict = {}
date_indices = None

# Process the file
result = read_file(file_name, data_dir)

if result is not None:
    COMID, dates, data = result
    data_dict[COMID] = {
        'dates': dates,
        'data': data
    }

    if date_indices is None:
        date_indices = dates

# Sort the dictionary by COMID
data_dict = OrderedDict(sorted(data_dict.items()))

# Output results
print(f"Final data_dict contains data from {len(data_dict)} COMID(s)")

Final data_dict contains data from 1 COMID(s)


In [4]:
# Define the sequence length
Lseq = 30  # Sequence length

# Define the training fraction
x = 0.7  # Percentage of data to be used for training

# Dictionaries to store training and testing days for each COMID
train_days = {}
test_days = {}

# Function to process a single COMID
def process_comid(comid, comid_data):
    dates = comid_data['dates']  # Corresponding ordinal dates
    flux = comid_data['data'][:, 0]  # Observed_Sediment_Flux is the first column of 'data'

    # Step 1: Find the rows where Observed_Sediment_Flux is not 'nan' and get the corresponding dates
    valid_days = [day for day in range(len(flux)) if not torch.isnan(flux[day])]
    valid_dates = [dates[day] for day in valid_days]

    # Step 2: Compute the training and testing split
    total_valid_count = len(valid_dates)
    target_training_count = int(x * total_valid_count)

    # Step 3: Split valid dates into training and testing sets
    training_dates = valid_dates[:target_training_count]
    testing_dates = valid_dates[target_training_count:]

    # Filter dates that correspond to indices greater than or equal to Lseq - 1
    training_dates_filtered = [date for idx, date in enumerate(training_dates) if valid_days[idx] >= (Lseq - 1)]
    testing_dates_filtered = [date for idx, date in enumerate(testing_dates) if valid_days[idx] >= (Lseq - 1)]

    return comid, training_dates_filtered, testing_dates_filtered

# Process COMIDs in parallel using ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    # Submit tasks for each COMID
    futures = {executor.submit(process_comid, comid, comid_data): comid for comid, comid_data in data_dict.items()}

    # Collect results as they complete
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing COMIDs"):
        comid, training_dates_filtered, testing_dates_filtered = future.result()
        train_days[comid] = training_dates_filtered
        test_days[comid] = testing_dates_filtered

# Final Summary
print(f"Processed {len(train_days)} COMIDs into train and test days.")

Processing COMIDs: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.34s/it]

Processed 1 COMIDs into train and test days.


In [5]:
# List to collect all training data across COMIDs
all_training_data = []

# Dictionary to store max valid_y_values for each COMID
comid_max_y_values = {}

# Iterate over each COMID in the data_dict
for comid, comid_data in tqdm(data_dict.items(), desc="Processing COMIDs"):
    # Get the last training day index from train_days
    last_training_date = train_days[comid][-1]  # Last training date
    last_training_index = np.where(comid_data['dates'] == last_training_date)[0][0]  # Index of the last training day

    # Extract the data up to the last training row
    training_data = comid_data['data'][:last_training_index + 1]  # All rows up to last_training_index

    # Separate inputs (excluding the first column)
    input_data = training_data[:, 1:]  # All columns except the first

    # Extract the first column (target values)
    y_values = training_data[:, 0]  # First column is the target

    # Filter out rows with 'nan' values for the target column
    valid_y_values = [y.item() for y in y_values if str(y.item()).strip().lower() != 'nan']

    # Compute and store the max of valid_y_values for the current COMID
    if valid_y_values:  # Ensure there are valid values
        comid_max_y_values[comid] = max(valid_y_values)  

    # Collect input data
    all_training_data.append(input_data)

# Combine all training data into a single array
all_training_data = np.vstack(all_training_data)

# Compute the overall mean and standard deviation for inputs
overall_mean = np.nanmean(all_training_data, axis=0)
overall_std = np.nanstd(all_training_data, axis=0)

# Apply the threshold to the standard deviation
overall_std[overall_std < 0.001] = 0.001

# Print results
print(f"Overall Mean (Inputs): {overall_mean}")
print(f"Overall Standard Deviation (Inputs): {overall_std}")

Processing COMIDs: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.31it/s]

Overall Mean (Inputs): [ 1.35882950e+02  1.46200012e+03  4.26106334e-01  3.45718674e-03
  3.04222514e-04  2.12073612e+00  1.15095437e+00  0.00000000e+00
  3.66467573e-02  9.85087872e+00  8.17691803e+00  6.57772903e+01
  4.02774841e-01  1.06712523e+01  4.20336097e-01  4.09233153e-01
  0.00000000e+00  6.67649433e-02  4.86338496e-01  1.79957043e+03
  2.90231567e+02  2.89522888e+02  2.89654968e+02  6.30249023e+02
  1.02759106e-04  5.34736589e-02  2.85216477e-02  0.00000000e+00
  1.50612640e+01  1.77915648e-01  2.05981421e+00  7.95171127e+01
  8.09491813e-01  2.29233503e+00  0.00000000e+00  7.06856226e-05
  0.00000000e+00  1.42251550e+06  1.47047016e+05  7.36038093e+10
  2.71730846e-06 -1.36232071e+01  4.88356600e+07  2.96302980e+07
  1.51104193e+01  3.99572074e-01  2.20545721e+00  1.21800000e+03
  8.19000000e+02  1.40100000e+03  1.39800000e+03  1.00800000e+03
  6.88000000e+02  4.03000000e+02  8.70000000e+01  5.90000000e+01
  6.80000000e+01  4.68000000e+02  1.07000000e+02  1.00000000e+01
  

In [6]:
class CustomData(Dataset):
    def __init__(self, data_dict, days, seq_len, mean_inputs, std_inputs, comid_max_y_values):
        """
        Args:
            data_dict: Dictionary containing 'data' and 'dates' for each COMID.
            days: Dictionary containing valid indices for each COMID.
            seq_len: Length of the input sequence.
            mean_inputs: Mean values for normalization of inputs.
            std_inputs: Standard deviation values for normalization of inputs.
            comid_max_y_values: Maximum target value for current COMID    
        """
        super(Dataset, self).__init__()
        self.data_dict = data_dict
        self.days = days
        self.seq_len = seq_len
        self.mean_inputs = mean_inputs
        self.std_inputs = std_inputs
        self.comid_max_y_values = comid_max_y_values
        self.indices = [
            (comid, torch.where(torch.tensor(self.data_dict[comid]['dates']) == day)[0].item())
            for comid, day_list in days.items() for day in day_list
        ]

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        """
        Args:
            idx: Index of the sample to retrieve.
        
        Returns:
            comid: The COMID of the sample.
            x: Normalized input sequence of shape (seq_len, num_features).
            y: Target value corresponding to the end of the sequence, normalized.
            y_date: The ordinal date of the target value y.       
        """
        comid, day_index = self.indices[idx]
        data = self.data_dict[comid]['data']
        dates = self.data_dict[comid]['dates']

        # Extract input sequence and target
        x = data[day_index - self.seq_len + 1:day_index + 1, 1:]  # Inputs
        y = data[day_index, 0]  # Target
        y_date = dates[day_index]  # Date of the target value

        # Normalize inputs
        x_normalized = (x - self.mean_inputs) / self.std_inputs

        # Define a small epsilon to handle zero target values
        epsilon = 1e-3

        # Check if y is 0 and add epsilon if so
        if y == 0:
            y = epsilon  # Avoid zero target values

        y = y.clone().detach().float() if isinstance(y, torch.Tensor) else torch.tensor(y, dtype=torch.float32)

        # Get the max y value for the current COMID
        max_y = self.comid_max_y_values.get(comid, 1.0)  # Default to 1.0 if not found

        # Normalize target by the COMID-specific max value
        y_normalized = y / max_y if max_y > 0 else y  # Avoid division by zero    

        # Return comid as a tensor as well
        comid_tensor = torch.tensor(comid, dtype=torch.long)

        # Return comid, normalized inputs, normalized target
        return comid_tensor, x_normalized, y_normalized, y_date

# Instantiate datasets
train_dataset = CustomData(
    data_dict=data_dict,
    days=train_days,
    seq_len=Lseq,
    mean_inputs=overall_mean,
    std_inputs=overall_std,
    comid_max_y_values=comid_max_y_values
)

test_dataset = CustomData(
    data_dict=data_dict,
    days=test_days,
    seq_len=Lseq,
    mean_inputs=overall_mean,
    std_inputs=overall_std,
    comid_max_y_values=comid_max_y_values
)

In [7]:
# Print the length of indices for both train and test datasets
train_indices_length = len(train_dataset.indices)
test_indices_length = len(test_dataset.indices)

# Output the lengths
print(f"Number of indices in train_dataset: {train_indices_length}")
print(f"Number of indices in test_dataset: {test_indices_length}")


Number of indices in train_dataset: 137
Number of indices in test_dataset: 60


In [8]:
# define LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = 0.40)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim, device = x.device).requires_grad_()
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim, device = x.device).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(self.dropout(hn[0,:,:]))
        return out

# Custom NSE computation function
def computeNSE(obs, pred):
    sse = np.sum((obs - pred)**2)
    sst = np.sum((obs - np.mean(obs))**2)
    nse = 1 - sse / sst
    return nse

In [9]:
# Set random seeds for reproducibility
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # Ensures consistency across GPUs
np.random.seed(seed)
random.seed(seed)

# Set device before using it in the dataset class
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

# Define model parameters
input_dim = len(overall_mean) # Number of features for the LSTM input
hidden_dim = 256  # Hidden dimension for LSTM
n_layers = 1  # Number of layers for LSTM
output_dim = 1  # Single target value output
learning_rate = 1e-4
epochs = 100
N = 2**5  # Adjust batch size as needed

# Initialize LSTM model
lstm_model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, n_layers=n_layers, output_dim=output_dim)
lstm_model = lstm_model.to(device)

# Define loss function and optimizer
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=learning_rate)

# Create DataLoader for training data
train_indices = list(range(len(train_dataset)))
train_sampler = SubsetRandomSampler(train_indices)
train_dataloader = DataLoader(train_dataset, batch_size=N, sampler=train_sampler)

# Training loop
model_state = []
for epoch in tqdm(range(epochs), desc="Training Progress"):
    lstm_model.train()  # Set model to training mode
    total_loss = 0.0  # Accumulate loss for the entire epoch
    all_predictions = []  # Store all predictions for the epoch
    all_observations = []  # Store all true values for the epoch    

    for batch_idx, (comid, x_batch, y_batch, y_date) in enumerate(train_dataloader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device).view(-1, 1)

        # Forward pass
        predictions = lstm_model(x_batch)  # Get predictions

        #print (predictions)
        loss = criterion(predictions, y_batch) # Compute loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        total_loss += loss.item()

        # Store predictions and true values for analysis
        all_predictions.append(predictions.detach().cpu())  # Move to CPU
        all_observations.append(y_batch.detach().cpu())  # Move to CPU

        # Clear memory
        del loss, predictions, x_batch, y_batch
        torch.cuda.empty_cache()

    # Calculate average loss for the epoch
    avg_loss = total_loss / len(train_dataloader)

    # Concatenate all predictions and observations
    all_predictions = torch.cat(all_predictions)
    all_observations = torch.cat(all_observations)

    # Compute additional metrics (e.g., NSE, RMSE)
    nse = computeNSE(all_observations.numpy(), all_predictions.numpy())
    rmse = torch.sqrt(torch.mean((1 - (all_predictions / all_observations)) ** 2)).item()

    # Print metrics
    print(f"Epoch {epoch + 1}/{epochs}, Avg Loss: {avg_loss:.4f}, NSE: {nse:.4f}, RMSE: {rmse:.4f}")

    # Save model state
    model_state.append(copy.deepcopy(lstm_model.state_dict()))

    # Clear memory
    del all_predictions, all_observations
    torch.cuda.empty_cache()

Using cuda device


Training Progress:   3%|█▉                                                              | 3/100 [00:00<00:10,  9.18it/s]

Epoch 1/100, Avg Loss: 0.2404, NSE: -1.2941, RMSE: 2.3086
Epoch 2/100, Avg Loss: 0.1973, NSE: -0.8437, RMSE: 2.0923
Epoch 3/100, Avg Loss: 0.2043, NSE: -0.6474, RMSE: 2.2857
Epoch 4/100, Avg Loss: 0.2028, NSE: -0.5875, RMSE: 2.8359


Training Progress:   7%|████▍                                                           | 7/100 [00:00<00:06, 13.30it/s]

Epoch 5/100, Avg Loss: 0.1836, NSE: -0.5117, RMSE: 3.2852
Epoch 6/100, Avg Loss: 0.1851, NSE: -0.3992, RMSE: 2.5230
Epoch 7/100, Avg Loss: 0.1696, NSE: -0.4180, RMSE: 2.5942
Epoch 8/100, Avg Loss: 0.1818, NSE: -0.3104, RMSE: 1.7452


...

Epoch 89/100, Avg Loss: 0.1084, NSE: 0.4416, RMSE: 1.3837
Epoch 90/100, Avg Loss: 0.1045, NSE: 0.4990, RMSE: 1.4233
Epoch 91/100, Avg Loss: 0.0889, NSE: 0.4855, RMSE: 1.2483
Epoch 92/100, Avg Loss: 0.0930, NSE: 0.4676, RMSE: 1.4373


Training Progress:  95%|███████████████████████████████████████████████████████████▊   | 95/100 [00:05<00:00, 19.39it/s]

Epoch 93/100, Avg Loss: 0.1062, NSE: 0.4819, RMSE: 1.7435
Epoch 94/100, Avg Loss: 0.0795, NSE: 0.5262, RMSE: 1.3355
Epoch 95/100, Avg Loss: 0.0962, NSE: 0.5021, RMSE: 1.2952
Epoch 96/100, Avg Loss: 0.0900, NSE: 0.4836, RMSE: 1.3632


Training Progress: 100%|██████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 16.92it/s]

Epoch 97/100, Avg Loss: 0.0986, NSE: 0.4688, RMSE: 1.3319
Epoch 98/100, Avg Loss: 0.0936, NSE: 0.4770, RMSE: 1.3020
Epoch 99/100, Avg Loss: 0.0895, NSE: 0.5529, RMSE: 1.2875
Epoch 100/100, Avg Loss: 0.0806, NSE: 0.5410, RMSE: 1.2376


In [10]:
# Custom sampler function 
class SubsetSampler(Sampler):
    def __init__(self, indices, generator=None):
        super(Sampler, self).__init__()
        self.indices = indices
        self.generator = generator

    def __iter__(self):
        for i in self.indices:
            yield i

    def __len__(self) -> int:
        return len(self.indices)

# Create DataLoader for testing data
test_indices = list(range(len(test_dataset)))  # Indices for all test data
test_sampler = SubsetSampler(test_indices)  # Using the custom SubsetSampler
test_dataloader = DataLoader(test_dataset, batch_size=N, sampler=test_sampler)

In [11]:
# Load the model state from the last epoch
last_epoch = len(model_state)  # Epochs start from 1
print(f"Loading Last Epoch: {last_epoch}")

last_state_dict = model_state[last_epoch - 1]  # Adjust for 0-based indexing
lstm_model.load_state_dict(last_state_dict, strict=True)

# Set model to evaluation mode
lstm_model.eval()

# Define the folder structure for saving testing results
base_path = "/N/lustre/project/proj-212/praval/Global_sediment_flux/lstm_model/codes/cluster/Hydrogeosciences_codes/"
folder_name = f"tr_{x}_bs_{N}_id_{input_dim}_hd_{hidden_dim}_nl_{n_layers}_lr_{learning_rate}_sl_{Lseq}_ep_{epochs}_se_{seed}_last_epoch"
testing_save_path = os.path.join(base_path, folder_name, "testing")

# Create directories if they don't exist
os.makedirs(testing_save_path, exist_ok=True)

# Collect predictions, observations, and COMIDs during testing
testing_results = []

with torch.no_grad():
    for batch_idx, (comid, x_batch, y_batch, y_date) in enumerate(test_dataloader):
        # Move inputs to device
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device).view(-1, 1)

        # Forward pass
        out = lstm_model(x_batch)

        # Collect predictions, true observations, and COMID
        for i in range(len(comid)):
            max_y = comid_max_y_values.get(comid[i].item(), 1.0)  # Default to 1.0 if not found
            observation = y_batch[i].item() * max_y
            prediction = max(out[i].item() * max_y, 0.001)  # Clamp negative values to zero
            date = datetime.date.fromordinal(int(y_date[i].item())).strftime('%m/%d/%Y')
            testing_results.append({
                "COMID": comid[i].item(),
                "Observation": observation,
                "Prediction": prediction,
                "Date": date 
            })

# Convert results to a DataFrame
testing_results_df = pd.DataFrame(testing_results)

# Save results to a CSV file
testing_results_file = os.path.join(testing_save_path, "testing_results.csv")
testing_results_df.to_csv(testing_results_file, index=False)

print(f"Testing results saved to: {testing_results_file}")


Loading Last Epoch: 100
Testing results saved to: /N/lustre/project/proj-212/praval/Global_sediment_flux/lstm_model/codes/cluster/Hydrogeosciences_codes/tr_0.7_bs_32_id_85_hd_256_nl_1_lr_0.0001_sl_30_ep_100_se_1_last_epoch/testing/testing_results.csv


In [12]:
# Save predictions and observations for the training dataset as well

# Define the folder structure for saving training results
training_save_path = os.path.join(base_path, folder_name, "training")

# Create directories if they don't exist
os.makedirs(training_save_path, exist_ok=True)

# Collect predictions, observations, and COMIDs during training evaluation
training_results = []

with torch.no_grad():
    for batch_idx, (comid, x_batch, y_batch, y_date) in enumerate(train_dataloader):
        # Move inputs to device
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device).view(-1, 1)

        # Forward pass
        out = lstm_model(x_batch)

        # Collect predictions, true observations, and COMID
        for i in range(len(comid)):
            max_y = comid_max_y_values.get(comid[i].item(), 1.0)  # Default to 1.0 if not found
            observation = y_batch[i].item() * max_y
            prediction = max(out[i].item() * max_y, 0.001)  # Clamp negative values to zero
            date = datetime.date.fromordinal(int(y_date[i].item())).strftime('%m/%d/%Y')
            training_results.append({
                "COMID": comid[i].item(),
                "Observation": observation,
                "Prediction": prediction,
                "Date": date 
            })

# Convert training results to a DataFrame
training_results_df = pd.DataFrame(training_results)

# Save training results to a CSV file
training_results_file = os.path.join(training_save_path, "training_results.csv")
training_results_df.to_csv(training_results_file, index=False)

print(f"Training results saved to: {training_results_file}")

Training results saved to: /N/lustre/project/proj-212/praval/Global_sediment_flux/lstm_model/codes/cluster/Hydrogeosciences_codes/tr_0.7_bs_32_id_85_hd_256_nl_1_lr_0.0001_sl_30_ep_100_se_1_last_epoch/training/training_results.csv
